In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import pandas as pd
import time

In [ ]:
# 브라우저 열기
driver = webdriver.Chrome()

# 스킨케어 데이터 수집

## 1. 올리브영 스킨케어 크롤링

In [ ]:
# 1. 스킨케어 랭킹 페이지 로드
driver.get("https://www.oliveyoung.co.kr/store/main/getBestList.do?dispCatNo=900000100100001&fltDispCatNo=10000010001&pageIdx=1&rowsPerPage=8&t_page=%EC%B9%B4%ED%85%8C%EA%B3%A0%EB%A6%AC%EA%B4%80&t_click=%EB%9E%AD%ED%82%B9BEST%EC%83%81%ED%92%88%EB%B8%8C%EB%9E%9C%EB%93%9C_%EC%9D%B8%EA%B8%B0%EC%83%81%ED%92%88%EB%8D%94%EB%B3%B4%EA%B8%B0")
time.sleep(3)  # 페이지 로드 대기

In [180]:
# 2. 페이지 스크롤
prev_height = driver.execute_script("return document.body.scrollHeight")

# 1위 - 100위
while True:
    # 스크롤 내리기
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)

    # 새로운 높이 가져오기
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == prev_height:
        break
    prev_height = new_height

In [ ]:
# 3-1. 랭킹 페이지 데이터 추출
titles = driver.find_elements(By.XPATH, '//*[@id="Container"]/div[2]/div[2]/ul/li/div/div/a/span')
products = driver.find_elements(By.XPATH, '//*[@id="Container"]/div[2]/div[2]/ul/li/div/div/a/p')
or_prices = driver.find_elements(By.XPATH, '//*[@id="Container"]/div[2]/div[2]/ul/li/div/p[1]/span[1]/span')
dis_prices = driver.find_elements(By.XPATH, '//*[@id="Container"]/div[2]/div[2]/ul/li/div/p[1]/span[2]/span')

print(len(titles))
print(len(products))
print(len(or_prices))
print(len(dis_prices)) # 세일 안하는 상품들 맞는지 더블 체크 필요

In [ ]:
# 4-1. 데이터 텍스트 변환
title_list = [t.text for t in titles]
print(len(title_list))
print(title_list[:5])

product_list = [p.text for p in products]
print(len(product_list))
print(product_list[:5])

or_price_list = [op.text for op in or_prices]
print(len(or_price_list))
print(or_price_list[:5])

100
100
100
97
100
['에스트라', '에스트라', '에스트라', '바이오힐보', '구달']
100
['[2월올영픽/1등크림] 에스트라 아토베리어365 크림 80ML 더블기획 (+세라히알 앰플 7ML+에이시카 세럼 3ML)', '[50%용량증정]에스트라 아토베리어365 세라-히알 속수분 앰플 30ML+15ML 기획', '[2월올영픽]에스트라 아토베리어365 로션 150ml 리필 기획 (+150ML 리필 파우치 + 에이시카 세럼 3ML)', '[2월 올영픽/PDRN 키트증정] 바이오힐보 프로바이오덤 콜라겐 리모델링 세럼 30ml 리필 기획 (+프로그램14ml)', '[세럼1위/6년연속수상/총 120ml 대용량]구달 청귤 비타C 잡티케어 세럼 70ml 대용량 어워즈 한정기획']
100
['59,400', '33,000', '53,400', '49,500', '50,000']


In [ ]:
# 4-1. 할인가 밀림 현상 방지 및 데이터 텍스트 변환
dis_price_list = []

for w in range(1, 26):
  for l in range(1, 5):
    
    try:
      dis = driver.find_element(By.XPATH, f'//*[@id="Container"]/div[2]/div[2]/ul[{w}]/li[{l}]/div/p[1]/span[2]/span')
      if dis.text:
         dis_price_list.append(dis.text)
      else:
        dis_price_list.append('') 
             
    except:
      dis_price_list.append('')
      
print(len(dis_price_list))
print(dis_price_list[:5])

100
['24,000', '28,900', '42,000', '24,500', '27,360']


In [197]:
# 3-2. 상품 상세 페이지 데이터 추출
wait = WebDriverWait(driver, 10)

main_classes = []
mid_classes = []
sub_classes = []
review_nums = []
review_scores = []

for w in range(1, 26):
  for l in range(1, 5):
    detail_page = driver.find_elements(By.XPATH, f'//*[@id="Container"]/div[2]/div[2]/ul[{w}]/li[{l}]/div/a')
    
    if detail_page:
        detail_page[0].click()
    else:
      continue

    main_class = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="midCatNm"]'))).text
    mid_class = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="smlCatNm"]'))).text
    sub_class = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="dtlCatNm"]'))).text 
    num = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="repReview"]/em'))).text
    score = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="repReview"]/b'))).text
  
    main_classes.append(main_class)
    mid_classes.append(mid_class)
    sub_classes.append(sub_class)
    review_nums.append(num)
    review_scores.append(score)

    driver.back()
    wait.until(EC.presence_of_element_located((By.XPATH, f'//*[@id="Container"]/div[2]/div[2]/ul[{w}]/li[{l}]/div/a')))


In [ ]:
# 4-2. 데이터 개수 확인
print(len(main_classes))
print(len(mid_classes))
print(len(sub_classes))
print(len(review_nums))
print(len(review_scores))

print(main_classes[0:5])
print(mid_classes[0:5])
print(sub_classes[0:5])
print(review_nums[0:5])
print(review_scores[0:5])

100
100
100
100
100
['스킨케어', '스킨케어', '스킨케어', '스킨케어', '스킨케어']
['크림', '에센스/세럼/앰플', '로션', '에센스/세럼/앰플', '에센스/세럼/앰플']
['크림', '에센스/세럼/앰플', '로션', '에센스/세럼/앰플', '에센스/세럼/앰플']
['(25,321건)', '(1,957건)', '(2,688건)', '(2,226건)', '(28,846건)']
['4.9', '4.8', '4.9', '4.7', '4.8']


In [3]:
# 5. 데이터 저장
olive_skincare_dic = {"brand": title_list, "product": product_list, "or_price": or_price_list, "dis_price": dis_price_list, "main_class": main_classes, "mid_class": mid_classes, "sub_class": sub_classes, "review_num": review_nums, "review_score": review_scores}

NameError: name 'title_list' is not defined

In [296]:
# DF 변환
olive_skincare_df = pd.DataFrame.from_dict(olive_skincare_dic, orient='index')
olive_skincare_df = olive_skincare_df.transpose()
olive_skincare_df.head()

,brand,product,or_price,dis_price,main_class,mid_class,sub_class,review_num,review_score
0,에스트라,[2월올영픽/1등크림] 에스트라 아토베리어365 크림 80ML 더블기획 (+세라히알...,"59,400","44,500",스킨케어,크림,크림,"(25,321건)",4.9
1,에스트라,[50%용량증정]에스트라 아토베리어365 세라-히알 속수분 앰플 30ML+15ML 기획,"33,000","24,700",스킨케어,에센스/세럼/앰플,에센스/세럼/앰플,"(1,957건)",4.8
2,에스트라,[2월올영픽]에스트라 아토베리어365 로션 150ml 리필 기획 (+150ML 리필...,"53,400","39,900",스킨케어,로션,로션,"(2,688건)",4.9
3,바이오힐보,[2월 올영픽/PDRN 키트증정] 바이오힐보 프로바이오덤 콜라겐 리모델링 세럼 30...,"49,500","27,300",스킨케어,에센스/세럼/앰플,에센스/세럼/앰플,"(2,226건)",4.7
4,구달,[세럼1위/6년연속수상/총 120ml 대용량]구달 청귤 비타C 잡티케어 세럼 70m...,"50,000","32,900",스킨케어,에센스/세럼/앰플,에센스/세럼/앰플,"(28,846건)",4.8


In [ ]:
# 할인가 정상적으로 들어갔는지 확인
print(len(dis_prices))
print(olive_skincare_df.loc[olive_skincare_df['dis_price']=='']['product'].nunique())
olive_skincare_df.loc[olive_skincare_df['dis_price']=='']

,brand,product,or_price,dis_price,main_class,mid_class,sub_class,review_num,review_score
31,아벤느,[장벽/보습]아벤느 트릭세라 뉴트리-플루이드 로션 200ml 더블 기획 *윤광 *광...,"44,000",,스킨케어,로션,로션,"(2,042건)",4.7
68,에스티로더,에스티로더 갈색병 30ml 기획 (갈색병 21ml 추가 증정),"127,000",,스킨케어,에센스/세럼/앰플,에센스/세럼/앰플,"(2,882건)",4.8
82,디오디너리,디오디너리 나이아신아마이드 10% + 징크 1% 30ml,"8,700",,스킨케어,에센스/세럼/앰플,에센스/세럼/앰플,"(10,438건)",4.6


In [386]:
# 인덱스 순으로 랭킹 컬럼 삽입 - 인덱스 컬럼 생성
olive_skincare_df.reset_index(inplace=True)
olive_skincare_df.head()

,index,brand,product,or_price,dis_price,main_class,mid_class,sub_class,review_num,review_score
0,0,에스트라,[2월올영픽/1등크림] 에스트라 아토베리어365 크림 80ML 더블기획 (+세라히알...,"59,400","44,500",스킨케어,크림,크림,"(25,321건)",4.9
1,1,에스트라,[50%용량증정]에스트라 아토베리어365 세라-히알 속수분 앰플 30ML+15ML 기획,"33,000","24,700",스킨케어,에센스/세럼/앰플,에센스/세럼/앰플,"(1,957건)",4.8
2,2,에스트라,[2월올영픽]에스트라 아토베리어365 로션 150ml 리필 기획 (+150ML 리필...,"53,400","39,900",스킨케어,로션,로션,"(2,688건)",4.9
3,3,바이오힐보,[2월 올영픽/PDRN 키트증정] 바이오힐보 프로바이오덤 콜라겐 리모델링 세럼 30...,"49,500","27,300",스킨케어,에센스/세럼/앰플,에센스/세럼/앰플,"(2,226건)",4.7
4,4,구달,[세럼1위/6년연속수상/총 120ml 대용량]구달 청귤 비타C 잡티케어 세럼 70m...,"50,000","32,900",스킨케어,에센스/세럼/앰플,에센스/세럼/앰플,"(28,846건)",4.8


In [387]:
# 인덱스 순으로 랭킹 컬럼 삽입 - 인덱스 컬럼 변경 및 값 변경
olive_skincare_df.rename(columns={'index':'ranking'}, inplace=True)
olive_skincare_df['ranking'] = olive_skincare_df['ranking'].apply(lambda x : x+1)
olive_skincare_df.head()

,ranking,brand,product,or_price,dis_price,main_class,mid_class,sub_class,review_num,review_score
0,1,에스트라,[2월올영픽/1등크림] 에스트라 아토베리어365 크림 80ML 더블기획 (+세라히알...,"59,400","44,500",스킨케어,크림,크림,"(25,321건)",4.9
1,2,에스트라,[50%용량증정]에스트라 아토베리어365 세라-히알 속수분 앰플 30ML+15ML 기획,"33,000","24,700",스킨케어,에센스/세럼/앰플,에센스/세럼/앰플,"(1,957건)",4.8
2,3,에스트라,[2월올영픽]에스트라 아토베리어365 로션 150ml 리필 기획 (+150ML 리필...,"53,400","39,900",스킨케어,로션,로션,"(2,688건)",4.9
3,4,바이오힐보,[2월 올영픽/PDRN 키트증정] 바이오힐보 프로바이오덤 콜라겐 리모델링 세럼 30...,"49,500","27,300",스킨케어,에센스/세럼/앰플,에센스/세럼/앰플,"(2,226건)",4.7
4,5,구달,[세럼1위/6년연속수상/총 120ml 대용량]구달 청귤 비타C 잡티케어 세럼 70m...,"50,000","32,900",스킨케어,에센스/세럼/앰플,에센스/세럼/앰플,"(28,846건)",4.8


In [ ]:
# 원본 데이터 저장
# olive_skincare_df.to_csv("oliveyoung_skincare_ranking.csv", index=False, encoding="utf-8-sig")

## 2. 올리브영 마스크팩 크롤링

In [310]:
# 1. 마스크팩 랭킹 페이지 로드
driver.get("https://www.oliveyoung.co.kr/store/main/getBestList.do?dispCatNo=900000100100001&fltDispCatNo=10000010009&pageIdx=1&rowsPerPage=8&t_page=%EB%9E%AD%ED%82%B9&t_click=%ED%8C%90%EB%A7%A4%EB%9E%AD%ED%82%B9_%EB%A7%88%EC%8A%A4%ED%81%AC%ED%8C%A9")
time.sleep(3)  # 페이지 로드 대기

In [311]:
# 2. 페이지 스크롤
prev_height = driver.execute_script("return document.body.scrollHeight")

In [312]:
# 1위 - 100위
while True:
    # 스크롤 내리기
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)

    # 새로운 높이 가져오기
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == prev_height:
        break
    prev_height = new_height

In [313]:
# 3-1. 랭킹 페이지 데이터 추출
titles = driver.find_elements(By.XPATH, '//*[@id="Container"]/div[2]/div[2]/ul/li/div/div/a/span')
products = driver.find_elements(By.XPATH, '//*[@id="Container"]/div[2]/div[2]/ul/li/div/div/a/p')
or_prices = driver.find_elements(By.XPATH, '//*[@id="Container"]/div[2]/div[2]/ul/li/div/p[1]/span[1]/span')
dis_prices = driver.find_elements(By.XPATH, '//*[@id="Container"]/div[2]/div[2]/ul/li/div/p[1]/span[2]/span')

print(len(titles))
print(len(products))
print(len(or_prices))
print(len(dis_prices)) # 세일 안하는 상품들 맞는지 더블 체크 필요

100
100
100
91


In [314]:
# 4-1. 데이터 텍스트로 변환
title_list = [t.text for t in titles]
print(title_list[0:5])

product_list = [p.text for p in products]
print(product_list[0:5])

or_price_list = [op.text for op in or_prices]
print(or_price_list[0:5])

# dis_price_list = [dis.text for dis in dis_prices]
# print(dis_price_list[0:5])

['더마토리', '더마토리', '더마토리', '메디힐', '메디힐']
['[리뉴얼] 더마토리 히알루론 수분샘 엠보 패드 60매+10매 기획', '[리뉴얼] 더마토리 시카 엑소좀 흔적 진정 거즈 패드 70매+10매 기획', '[인씨 PICK/리뉴얼] 더마토리 살리시닉 포어 클리어 블랙 패드 70매+10매 기획 세트', '[51관왕/200매] 메디힐 네모패드 100+100매 더블기획 7종 골라담기', '[13년연속 1위] 메디힐 에센셜 마스크팩 1매 고기능 7종 택1']
['26,000', '26,000', '26,000', '39,900', '2,000']


In [315]:
# 4-1. 할인가 밀림 현상 방지 및 데이터 텍스트 변환
dis_price_list = []

for w in range(1, 26):
  for l in range(1, 5):
    
    try:
      dis = driver.find_element(By.XPATH, f'//*[@id="Container"]/div[2]/div[2]/ul[{w}]/li[{l}]/div/p[1]/span[2]/span')
      if dis.text:
         dis_price_list.append(dis.text)
      else:
        dis_price_list.append('') 
             
    except:
      dis_price_list.append('')
      
print(len(dis_price_list))
print(dis_price_list[:5])

100
['17,600', '17,600', '17,600', '28,900', '1,000']


In [316]:
# 3-2. 상품 상세 페이지 데이터 추출
wait = WebDriverWait(driver, 10)

main_classes = []
mid_classes = []
sub_classes = []
review_nums = []
review_scores = []

for w in range(1, 26):
  for l in range(1, 5):
    detail_page = driver.find_elements(By.XPATH, f'//*[@id="Container"]/div[2]/div[2]/ul[{w}]/li[{l}]/div/a')
    
    if detail_page:
        detail_page[0].click()
    else:
      continue

    main_class = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="midCatNm"]'))).text
    mid_class = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="smlCatNm"]'))).text
    sub_class = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="dtlCatNm"]'))).text 
    num = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="repReview"]/em'))).text
    score = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="repReview"]/b'))).text
  
    main_classes.append(main_class)
    mid_classes.append(mid_class)
    sub_classes.append(sub_class)
    review_nums.append(num)
    review_scores.append(score)

    driver.back()
    wait.until(EC.presence_of_element_located((By.XPATH, f'//*[@id="Container"]/div[2]/div[2]/ul[{w}]/li[{l}]/div/a')))


In [317]:
# 4-2. 데이터 개수 확인
print(len(main_classes))
print(len(mid_classes))
print(len(sub_classes))
print(len(review_nums))
print(len(review_scores))

print(main_classes[0:5])
print(mid_classes[0:5])
print(sub_classes[0:5])
print(review_nums[0:5])
print(review_scores[0:5])

100
100
100
100
100
['마스크팩', '마스크팩', '마스크팩', '마스크팩', '마스크팩']
['패드', '패드', '패드', '패드', '시트팩']
['패드', '패드', '패드', '패드', '시트팩']
['(8건)', '(14건)', '(143건)', '(26,581건)', '(229,962건)']
['4.6', '4.9', '4.9', '4.7', '4.9']


In [318]:
# 5. 데이터 저장
olive_mask_dic = {"brand": title_list, "product": product_list, "or_price": or_price_list, "dis_price": dis_price_list, "main_class": main_classes, "mid_class": mid_classes, "sub_class": sub_classes, "review_num": review_nums, "review_score": review_scores}

In [319]:
# DF 변환
olive_mask_df = pd.DataFrame.from_dict(olive_mask_dic, orient='index')
olive_mask_df = olive_mask_df.transpose()
olive_mask_df.head()

,brand,product,or_price,dis_price,main_class,mid_class,sub_class,review_num,review_score
0,더마토리,[리뉴얼] 더마토리 히알루론 수분샘 엠보 패드 60매+10매 기획,"26,000","17,600",마스크팩,패드,패드,(8건),4.6
1,더마토리,[리뉴얼] 더마토리 시카 엑소좀 흔적 진정 거즈 패드 70매+10매 기획,"26,000","17,600",마스크팩,패드,패드,(14건),4.9
2,더마토리,[인씨 PICK/리뉴얼] 더마토리 살리시닉 포어 클리어 블랙 패드 70매+10매 기...,"26,000","17,600",마스크팩,패드,패드,(143건),4.9
3,메디힐,[51관왕/200매] 메디힐 네모패드 100+100매 더블기획 7종 골라담기,"39,900","28,900",마스크팩,패드,패드,"(26,581건)",4.7
4,메디힐,[13년연속 1위] 메디힐 에센셜 마스크팩 1매 고기능 7종 택1,"2,000","1,000",마스크팩,시트팩,시트팩,"(229,962건)",4.9


In [327]:
# 할인가 정상적으로 들어갔는지 확인
print(len(dis_prices))
print(olive_mask_df.loc[olive_mask_df['dis_price']=='']['product'].nunique())
olive_mask_df.loc[olive_mask_df['dis_price']=='']

91
9


,brand,product,or_price,dis_price,main_class,mid_class,sub_class,review_num,review_score
33,스킨푸드,[700만개 판매/단독기획] 스킨푸드 캐롯 카로틴 카밍 워터 패드 기획 (60매+1...,"26,000",,마스크팩,패드,패드,"(61,247건)",4.8
41,비플레인,[3분/모공각질진정팩] 비플레인 녹두 모공 클레이 팩 120ml (+12ml*4ea...,"24,000",,마스크팩,페이셜팩,워시오프팩,"(11,248건)",4.7
60,브링그린,브링그린 프레시마스크 1매 5종,"1,800",,마스크팩,시트팩,시트팩,"(45,304건)",4.8
62,브링그린,[1+1] 브링그린 티트리시카포어코팩 5매입,"11,800",,마스크팩,코팩,코팩,"(10,227건)",4.8
68,메디필,메디필 랩핑 마스크팩 70mL 브러시 기획세트 2종 택1(레드락토 콜라겐/히알루론산...,"25,000",,마스크팩,페이셜팩,필오프팩,"(6,548건)",4.7
69,유세린,[데일리진정장벽] 유세린 울트라 센시티브 리페어 마스크팩 5매,"30,000",,마스크팩,시트팩,시트팩,"(1,802건)",4.8
74,메디힐,[26관왕/1위] 메디힐 더마 모델링 팩 풀패키지 5종 골라담기,"5,000",,마스크팩,페이셜팩,모델링팩,"(16,912건)",4.8
89,메디앤서,[모공쿨링/대용량]메디앤서 포어 콜라겐 나노샷 마스크 10매,"60,000",,마스크팩,시트팩,시트팩,(16건),4.4
90,에스트라,[NEW] 에스트라 에이시카365 쿨링팩패드 pH4.5 60매,"28,000",,마스크팩,패드,패드,(5건),5.0


In [362]:
# 인덱스 순으로 랭킹 컬럼 삽입 - 인덱스 컬럼 생성
olive_mask_df.reset_index(inplace=True)
olive_mask_df.head()

,index,brand,product,or_price,dis_price,main_class,mid_class,sub_class,review_num,review_score
0,0,더마토리,[리뉴얼] 더마토리 히알루론 수분샘 엠보 패드 60매+10매 기획,"26,000","17,600",마스크팩,패드,패드,(8건),4.6
1,1,더마토리,[리뉴얼] 더마토리 시카 엑소좀 흔적 진정 거즈 패드 70매+10매 기획,"26,000","17,600",마스크팩,패드,패드,(14건),4.9
2,2,더마토리,[인씨 PICK/리뉴얼] 더마토리 살리시닉 포어 클리어 블랙 패드 70매+10매 기...,"26,000","17,600",마스크팩,패드,패드,(143건),4.9
3,3,메디힐,[51관왕/200매] 메디힐 네모패드 100+100매 더블기획 7종 골라담기,"39,900","28,900",마스크팩,패드,패드,"(26,581건)",4.7
4,4,메디힐,[13년연속 1위] 메디힐 에센셜 마스크팩 1매 고기능 7종 택1,"2,000","1,000",마스크팩,시트팩,시트팩,"(229,962건)",4.9


In [363]:
# 인덱스 순으로 랭킹 컬럼 삽입 - 인덱스 컬럼 변경 및 값 변경
olive_mask_df.rename(columns={'index':'ranking'}, inplace=True)
olive_mask_df['ranking'] = olive_mask_df['ranking'].apply(lambda x : x+1)
olive_mask_df.head()

,ranking,brand,product,or_price,dis_price,main_class,mid_class,sub_class,review_num,review_score
0,1,더마토리,[리뉴얼] 더마토리 히알루론 수분샘 엠보 패드 60매+10매 기획,"26,000","17,600",마스크팩,패드,패드,(8건),4.6
1,2,더마토리,[리뉴얼] 더마토리 시카 엑소좀 흔적 진정 거즈 패드 70매+10매 기획,"26,000","17,600",마스크팩,패드,패드,(14건),4.9
2,3,더마토리,[인씨 PICK/리뉴얼] 더마토리 살리시닉 포어 클리어 블랙 패드 70매+10매 기...,"26,000","17,600",마스크팩,패드,패드,(143건),4.9
3,4,메디힐,[51관왕/200매] 메디힐 네모패드 100+100매 더블기획 7종 골라담기,"39,900","28,900",마스크팩,패드,패드,"(26,581건)",4.7
4,5,메디힐,[13년연속 1위] 메디힐 에센셜 마스크팩 1매 고기능 7종 택1,"2,000","1,000",마스크팩,시트팩,시트팩,"(229,962건)",4.9


In [ ]:
# 원본 데이터 저장
# olive_mask_df.to_csv("oliveyoung_mask_ranking.csv", index=False, encoding="utf-8-sig")

## 3. 올리브영 클렌징 크롤링

In [329]:
# 1. 클렌징 랭킹 페이지 로드
driver.get("https://www.oliveyoung.co.kr/store/main/getBestList.do?dispCatNo=900000100100001&fltDispCatNo=10000010010&pageIdx=1&rowsPerPage=8&t_page=%EB%9E%AD%ED%82%B9&t_click=%ED%8C%90%EB%A7%A4%EB%9E%AD%ED%82%B9_%ED%81%B4%EB%A0%8C%EC%A7%95")
time.sleep(3)  # 페이지 로드 대기

In [330]:
# 2. 페이지 스크롤
prev_height = driver.execute_script("return document.body.scrollHeight")

# 1위 - 100위
while True:
    # 스크롤 내리기
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)

    # 새로운 높이 가져오기
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == prev_height:
        break
    prev_height = new_height

In [331]:
# 3-1. 랭킹 페이지 데이터 추출
titles = driver.find_elements(By.XPATH, '//*[@id="Container"]/div[2]/div[2]/ul/li/div/div/a/span')
products = driver.find_elements(By.XPATH, '//*[@id="Container"]/div[2]/div[2]/ul/li/div/div/a/p')
or_prices = driver.find_elements(By.XPATH, '//*[@id="Container"]/div[2]/div[2]/ul/li/div/p[1]/span[1]/span')
dis_prices = driver.find_elements(By.XPATH, '//*[@id="Container"]/div[2]/div[2]/ul/li/div/p[1]/span[2]/span')

# 4-1. 데이터 개수 확인
print(len(titles))
print(len(products))
print(len(or_prices))
print(len(dis_prices)) # 세일 안하는 상품들 맞는지 더블 체크 필요

100
100
100
88


In [332]:
# 4-1. 데이터 텍스트로 변환
title_list = [t.text for t in titles]
print(title_list[0:5])

product_list = [p.text for p in products]
print(product_list[0:5])

or_price_list = [op.text for op in or_prices]
print(or_price_list[0:5])

# dis_price_list = [dis.text for dis in dis_prices]
# print(dis_price_list[0:5])

['휩드', '마녀공장', '바이오더마', '비플레인', '바닐라코']
['[2월 올영픽] 휩드 비건 팩클렌저 130g 기획/단품 (무화버터/머그트리/유자몽)', '[4년연속1위]마녀공장 퓨어 클렌징 오일 200mlX2 더블기획', '[2월 올영픽]바이오더마 센시비오 H2O 850ml 기획 (+K2 콜라보 폴딩 에코백)', '[한정수량] 비플레인 녹두 약산성 클렌징폼 160ml + 40ml 기획', '[1등 클렌징밤] 바닐라코 클린잇제로 오리지널 클렌징밤 100ml 기획 (+미니어처키트 7ml*4개입)']
['29,000', '37,000', '37,500', '21,900', '20,000']


In [333]:
# 4-1. 할인가 밀림 현상 방지 및 데이터 텍스트 변환
dis_price_list = []

for w in range(1, 26):
  for l in range(1, 5):
    
    try:
      dis = driver.find_element(By.XPATH, f'//*[@id="Container"]/div[2]/div[2]/ul[{w}]/li[{l}]/div/p[1]/span[2]/span')
      if dis.text:
         dis_price_list.append(dis.text)
      else:
        dis_price_list.append('') 
             
    except:
      dis_price_list.append('')
      
print(len(dis_price_list))
print(dis_price_list[:5])


100
['22,900', '25,900', '22,900', '18,600', '14,800']


In [334]:
# 3-2. 상품 상세 페이지 데이터 추출
wait = WebDriverWait(driver, 10)

main_classes = []
mid_classes = []
sub_classes = []
review_nums = []
review_scores = []

for w in range(1, 26):
  for l in range(1, 5):
    detail_page = driver.find_elements(By.XPATH, f'//*[@id="Container"]/div[2]/div[2]/ul[{w}]/li[{l}]/div/a')
    
    if detail_page:
        detail_page[0].click()
    else:
      continue

    main_class = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="midCatNm"]'))).text
    mid_class = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="smlCatNm"]'))).text
    sub_class = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="dtlCatNm"]'))).text 
    num = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="repReview"]/em'))).text
    score = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="repReview"]/b'))).text
  
    main_classes.append(main_class)
    mid_classes.append(mid_class)
    sub_classes.append(sub_class)
    review_nums.append(num)
    review_scores.append(score)

    driver.back()
    wait.until(EC.presence_of_element_located((By.XPATH, f'//*[@id="Container"]/div[2]/div[2]/ul[{w}]/li[{l}]/div/a')))

In [335]:
# 4-2. 데이터 개수 확인
print(len(main_classes))
print(len(mid_classes))
print(len(sub_classes))
print(len(review_nums))
print(len(review_scores))

print(main_classes[0:5])
print(mid_classes[0:5])
print(sub_classes[0:5])
print(review_nums[0:5])
print(review_scores[0:5])

100
100
100
100
100
['클렌징', '클렌징', '클렌징', '클렌징', '클렌징']
['클렌징폼/젤', '오일/밤', '워터/밀크', '클렌징폼/젤', '오일/밤']
['클렌징폼/젤', '클렌징오일', '클렌징워터', '클렌징폼/젤', '클렌징밤']
['(2,162건)', '(13,929건)', '(82,075건)', '(10,743건)', '(4,121건)']
['4.8', '4.9', '4.9', '4.8', '4.9']


In [336]:
# 5. 데이터 저장
olive_clean_dic = {"brand": title_list, "product": product_list, "or_price": or_price_list, "dis_price": dis_price_list, "main_class": main_classes, "mid_class": mid_classes, "sub_class": sub_classes, "review_num": review_nums, "review_score": review_scores}

In [343]:
# DF 변환
olive_clean_df = pd.DataFrame.from_dict(olive_clean_dic, orient='index')
olive_clean_df = olive_clean_df.transpose()
olive_clean_df.head()

,brand,product,or_price,dis_price,main_class,mid_class,sub_class,review_num,review_score
0,휩드,[2월 올영픽] 휩드 비건 팩클렌저 130g 기획/단품 (무화버터/머그트리/유자몽),"29,000","22,900",클렌징,클렌징폼/젤,클렌징폼/젤,"(2,162건)",4.8
1,마녀공장,[4년연속1위]마녀공장 퓨어 클렌징 오일 200mlX2 더블기획,"37,000","25,900",클렌징,오일/밤,클렌징오일,"(13,929건)",4.9
2,바이오더마,[2월 올영픽]바이오더마 센시비오 H2O 850ml 기획 (+K2 콜라보 폴딩 에코백),"37,500","22,900",클렌징,워터/밀크,클렌징워터,"(82,075건)",4.9
3,비플레인,[한정수량] 비플레인 녹두 약산성 클렌징폼 160ml + 40ml 기획,"21,900","18,600",클렌징,클렌징폼/젤,클렌징폼/젤,"(10,743건)",4.8
4,바닐라코,[1등 클렌징밤] 바닐라코 클린잇제로 오리지널 클렌징밤 100ml 기획 (+미니어처...,"20,000","14,800",클렌징,오일/밤,클렌징밤,"(4,121건)",4.9


In [338]:
# 할인가 정상적으로 들어갔는지 확인
print(len(dis_prices))
print(olive_clean_df.loc[olive_clean_df['dis_price']=='']['product'].nunique())
olive_clean_df.loc[olive_clean_df['dis_price']=='']

88
12


,brand,product,or_price,dis_price,main_class,mid_class,sub_class,review_num,review_score
45,폰즈,[나띠 PICK] 폰즈 클리어 스파 립앤아이 리무버 300ml,"24,500",,클렌징,립&아이리무버,립&아이리무버,"(81,629건)",4.9
50,메디필,[모공/약알칼리성] 메디필 레드 락토 콜라겐 클리어 폼 2.0 기획 (300ml+딥...,"24,000",,클렌징,클렌징폼/젤,클렌징폼/젤,"(16,578건)",4.7
51,메이크프렘,메이크프렘 세이프 미 릴리프 모이스처 클렌징밀크 200ml 기획(+100ml 리필),"28,000",,클렌징,워터/밀크,클렌징밀크/크림,"(7,137건)",4.8
52,비오텀,비오텀 클렌징 밀크 200ml,"27,800",,클렌징,워터/밀크,클렌징밀크/크림,"(3,207건)",4.8
53,아크네스,[촉촉클렌저/거품] 아크네스 포밍워시 200㎖,"16,500",,클렌징,클렌징폼/젤,클렌징폼/젤,"(3,510건)",4.7
69,메디필,[트러블/약산성] 메디필 그린 시카 콜라겐 클리어 폼 2.0 기획,"24,000",,클렌징,클렌징폼/젤,클렌징폼/젤,"(4,832건)",4.8
75,런드리유,[모공브러쉬 증정/단독기획] 런드리유 클린 페이스 가글 스트롱 젤 투 폼 클렌저 기...,"22,000",,클렌징,클렌징폼/젤,클렌징폼/젤,"(11,351건)",4.9
78,일소,[2024 어워즈] 일소 슈퍼 멜팅 세범 소프트너 150ml 한정기획 (+솜40매+...,"28,900",,클렌징,필링&스크럽,피지클리너,"(13,429건)",4.7
79,브링그린,[NEW] 브링그린 티트리시카딥클렌징폼 120mL,"10,900",,클렌징,클렌징폼/젤,클렌징폼/젤,(149건),4.8
87,이즈앤트리,[단독기획] 이즈앤트리 참마 비건 밀크 클렌저 220ml 리필 기획 (+리필100ml),"25,000",,클렌징,워터/밀크,클렌징밀크/크림,"(4,262건)",4.8


In [ ]:
# 인덱스 순으로 랭킹 컬럼 삽입 - 인덱스 컬럼 생성
olive_clean_df.reset_index(inplace=True)
olive_clean_df.head()

,index,brand,product,or_price,dis_price,main_class,mid_class,sub_class,review_num,review_score
0,0,휩드,[2월 올영픽] 휩드 비건 팩클렌저 130g 기획/단품 (무화버터/머그트리/유자몽),"29,000","22,900",클렌징,클렌징폼/젤,클렌징폼/젤,"(2,162건)",4.8
1,1,마녀공장,[4년연속1위]마녀공장 퓨어 클렌징 오일 200mlX2 더블기획,"37,000","25,900",클렌징,오일/밤,클렌징오일,"(13,929건)",4.9
2,2,바이오더마,[2월 올영픽]바이오더마 센시비오 H2O 850ml 기획 (+K2 콜라보 폴딩 에코백),"37,500","22,900",클렌징,워터/밀크,클렌징워터,"(82,075건)",4.9
3,3,비플레인,[한정수량] 비플레인 녹두 약산성 클렌징폼 160ml + 40ml 기획,"21,900","18,600",클렌징,클렌징폼/젤,클렌징폼/젤,"(10,743건)",4.8
4,4,바닐라코,[1등 클렌징밤] 바닐라코 클린잇제로 오리지널 클렌징밤 100ml 기획 (+미니어처...,"20,000","14,800",클렌징,오일/밤,클렌징밤,"(4,121건)",4.9


In [345]:
# 인덱스 순으로 랭킹 컬럼 삽입 - 인덱스 컬럼 변경 및 값 변경
olive_clean_df.rename(columns={'index':'ranking'}, inplace=True)
olive_clean_df['ranking'] = olive_clean_df['ranking'].apply(lambda x : x+1)
olive_clean_df.head()

,ranking,brand,product,or_price,dis_price,main_class,mid_class,sub_class,review_num,review_score
0,1,휩드,[2월 올영픽] 휩드 비건 팩클렌저 130g 기획/단품 (무화버터/머그트리/유자몽),"29,000","22,900",클렌징,클렌징폼/젤,클렌징폼/젤,"(2,162건)",4.8
1,2,마녀공장,[4년연속1위]마녀공장 퓨어 클렌징 오일 200mlX2 더블기획,"37,000","25,900",클렌징,오일/밤,클렌징오일,"(13,929건)",4.9
2,3,바이오더마,[2월 올영픽]바이오더마 센시비오 H2O 850ml 기획 (+K2 콜라보 폴딩 에코백),"37,500","22,900",클렌징,워터/밀크,클렌징워터,"(82,075건)",4.9
3,4,비플레인,[한정수량] 비플레인 녹두 약산성 클렌징폼 160ml + 40ml 기획,"21,900","18,600",클렌징,클렌징폼/젤,클렌징폼/젤,"(10,743건)",4.8
4,5,바닐라코,[1등 클렌징밤] 바닐라코 클린잇제로 오리지널 클렌징밤 100ml 기획 (+미니어처...,"20,000","14,800",클렌징,오일/밤,클렌징밤,"(4,121건)",4.9


In [ ]:
# 원본 데이터 저장
# olive_clean_df.to_csv("oliveyoung_cleansing_ranking.csv", index=False, encoding="utf-8-sig")

# 메이크업 데이터 수집

## 1. 올리브영 선케어 크롤링

In [347]:
# 선케어 랭킹 페이지 로드
driver.get("https://www.oliveyoung.co.kr/store/main/getBestList.do?dispCatNo=900000100100001&fltDispCatNo=10000010011&pageIdx=1&rowsPerPage=8&t_page=%EB%9E%AD%ED%82%B9&t_click=%ED%8C%90%EB%A7%A4%EB%9E%AD%ED%82%B9_%EC%84%A0%EC%BC%80%EC%96%B4")
time.sleep(3)  # 페이지 로드 대기

In [348]:
# 2. 페이지 스크롤
prev_height = driver.execute_script("return document.body.scrollHeight")

# 1위 - 100위
while True:
    # 스크롤 내리기
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)

    # 새로운 높이 가져오기
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == prev_height:
        break
    prev_height = new_height

In [349]:
# 3-1. 랭킹 페이지 데이터 추출출
titles = driver.find_elements(By.XPATH, '//*[@id="Container"]/div[2]/div[2]/ul/li/div/div/a/span')
products = driver.find_elements(By.XPATH, '//*[@id="Container"]/div[2]/div[2]/ul/li/div/div/a/p')
or_prices = driver.find_elements(By.XPATH, '//*[@id="Container"]/div[2]/div[2]/ul/li/div/p[1]/span[1]/span')
dis_prices = driver.find_elements(By.XPATH, '//*[@id="Container"]/div[2]/div[2]/ul/li/div/p[1]/span[2]/span')

print(len(titles))
print(len(products))
print(len(or_prices))
print(len(dis_prices)) # 세일 안하는 상품들 맞는지 더블 체크 필요

100
100
100
76


In [350]:
# 4-2. 데이터 텍스트로 변환
title_list = [t.text for t in titles]
print(title_list[0:5])

product_list = [p.text for p in products]
print(product_list[0:5])

or_price_list = [op.text for op in or_prices]
print(or_price_list[0:5])

# dis_price_list = [dis.text for dis in dis_prices]
# print(dis_price_list[0:5])

['달바', '이지듀', '구달', '라운드랩', '라로슈포제']
['[물광자차/핑크광채썬] 달바 핑크 톤업 선크림 듀오 기획 (50ml+50ml)', '[한가인PICK/기미커버개선] 이지듀 기미 앰플 쿠션 15g', '[1+1기획] 구달 맑은 어성초 진정 수분 선크림 50ml 1+1기획', '[1등썬/수분밀착] 라운드랩 자작나무 수분 선크림 50ml 기획 (+클렌저 20ml)', '[2월 올영픽/올영 단독] 라로슈포제 시카플라스트 리페어 선크림 40ml 기획 (+시카토너50ml+시카밤3ml 증정)']
['51,000', '39,800', '20,000', '25,000', '34,000']


In [351]:
# 4-1. 할인가 밀림 현상 방지 및 데이터 텍스트 변환
dis_price_list = []

for w in range(1, 26):
  for l in range(1, 5):
    
    try:
      dis = driver.find_element(By.XPATH, f'//*[@id="Container"]/div[2]/div[2]/ul[{w}]/li[{l}]/div/p[1]/span[2]/span')
      if dis.text:
         dis_price_list.append(dis.text)
      else:
        dis_price_list.append('') 
             
    except:
      dis_price_list.append('')
      
print(len(dis_price_list))
print(dis_price_list[:5])

100
['35,700', '36,900', '17,500', '17,500', '25,000']


In [352]:
# 3-2. 상품 상세 페이지 데이터 추출
wait = WebDriverWait(driver, 10)

main_classes = []
mid_classes = []
sub_classes = []
review_nums = []
review_scores = []

for w in range(1, 26):
  for l in range(1, 5):
    detail_page = driver.find_elements(By.XPATH, f'//*[@id="Container"]/div[2]/div[2]/ul[{w}]/li[{l}]/div/a')
    
    if detail_page:
        detail_page[0].click()
    else:
      continue

    main_class = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="midCatNm"]'))).text
    mid_class = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="smlCatNm"]'))).text
    sub_class = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="dtlCatNm"]'))).text 
    num = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="repReview"]/em'))).text
    score = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="repReview"]/b'))).text
  
    main_classes.append(main_class)
    mid_classes.append(mid_class)
    sub_classes.append(sub_class)
    review_nums.append(num)
    review_scores.append(score)

    driver.back()
    wait.until(EC.presence_of_element_located((By.XPATH, f'//*[@id="Container"]/div[2]/div[2]/ul[{w}]/li[{l}]/div/a')))

In [353]:
# 4-2. 데이터 개수 확인
print(len(main_classes))
print(len(mid_classes))
print(len(sub_classes))
print(len(review_nums))
print(len(review_scores))

print(main_classes[0:5])
print(mid_classes[0:5])
print(sub_classes[0:5])
print(review_nums[0:5])
print(review_scores[0:5])

100
100
100
100
100
['선케어', '선케어', '선케어', '선케어', '선케어']
['선크림', '선쿠션', '선크림', '선크림', '선크림']
['선크림', '선쿠션', '선크림', '선크림', '선크림']
['(7,408건)', '(175건)', '(28,908건)', '(14,207건)', '(137건)']
['4.7', '4.6', '4.8', '4.8', '4.7']


In [354]:
# 5. 데이터 저장
olive_suncare_dic = {"brand": title_list, "product": product_list, "or_price": or_price_list, "dis_price": dis_price_list, "main_class": main_classes, "mid_class": mid_classes, "sub_class": sub_classes, "review_num": review_nums, "review_score": review_scores}

In [366]:
# DF 변환
olive_suncare_df = pd.DataFrame.from_dict(olive_suncare_dic, orient='index')
olive_suncare_df = olive_suncare_df.transpose()
olive_suncare_df.head()

,brand,product,or_price,dis_price,main_class,mid_class,sub_class,review_num,review_score
0,달바,[물광자차/핑크광채썬] 달바 핑크 톤업 선크림 듀오 기획 (50ml+50ml),"51,000","35,700",선케어,선크림,선크림,"(7,408건)",4.7
1,이지듀,[한가인PICK/기미커버개선] 이지듀 기미 앰플 쿠션 15g,"39,800","36,900",선케어,선쿠션,선쿠션,(175건),4.6
2,구달,[1+1기획] 구달 맑은 어성초 진정 수분 선크림 50ml 1+1기획,"20,000","17,500",선케어,선크림,선크림,"(28,908건)",4.8
3,라운드랩,[1등썬/수분밀착] 라운드랩 자작나무 수분 선크림 50ml 기획 (+클렌저 20ml),"25,000","17,500",선케어,선크림,선크림,"(14,207건)",4.8
4,라로슈포제,[2월 올영픽/올영 단독] 라로슈포제 시카플라스트 리페어 선크림 40ml 기획 (+...,"34,000","25,000",선케어,선크림,선크림,(137건),4.7


In [367]:
# 할인가 정상적으로 들어갔는지 확인
print(len(dis_prices))
print(olive_suncare_df.loc[olive_suncare_df['dis_price']=='']['product'].nunique())
olive_suncare_df.loc[olive_suncare_df['dis_price']=='']

76
24


,brand,product,or_price,dis_price,main_class,mid_class,sub_class,review_num,review_score
9,넘버즈인,[더블기획] 넘버즈인 3번 도자기결 파데스킵 톤업베이지 35ml+35ml (SPF5...,"26,000",,선케어,선크림,선크림,"(10,772건)",4.6
14,넘버즈인,[1+1 기획] 넘버즈인 1번 청초 진정맑은 물막선크림 50ml+50ml 듀오기획 ...,"26,000",,선케어,선크림,선크림,"(5,105건)",4.8
24,구달,[고윤정PICK/1+1기획] 구달 맑은 어성초 진정 수분 톤업 선크림 50ml 1+1기획,"22,000",,선케어,선크림,선크림,"(4,016건)",4.6
28,셀퓨전씨,[UV에이징케어/잡티톤업썬] 셀퓨전씨 잡티 토닝 썬스크린 35ml 더블기획,"27,900",,선케어,선크림,선크림,(129건),4.9
33,달바,[수분자차/촉촉보습썬] 달바 에센스 선크림 듀오 기획 (50ml+50ml),"51,000",,선케어,선크림,선크림,"(7,178건)",4.8
36,셀퓨전씨,[UV에이징케어/밀착차단] 셀퓨전씨 레이저 UV 썬스크린 35ml 1+1 기획세트,"29,400",,선케어,선크림,선크림,"(2,452건)",4.8
44,브링그린,브링그린 티트리시카톤업선쿠션 15g_NEW,"20,000",,선케어,선쿠션,선쿠션,"(1,862건)",4.6
45,브링그린,브링그린 티트리시카쿨링선스틱 20g_NEW,"20,000",,선케어,선스틱,선스틱,(731건),4.6
47,스킨1004,[수분 선크림]스킨1004 마다가스카르 센텔라 히알루-시카 워터핏 선세럼 50ml,"19,000",,선케어,선크림,선크림,"(3,501건)",4.9
51,에스트라,에스트라 더마UV365 장벽수분 무기자차 선크림 40ml+20ml 기획,"31,000",,선케어,선크림,선크림,"(3,545건)",4.8


In [368]:
# 인덱스 순으로 랭킹 컬럼 삽입 - 인덱스 컬럼 생성
olive_suncare_df.reset_index(inplace=True)
olive_suncare_df.head()

,index,brand,product,or_price,dis_price,main_class,mid_class,sub_class,review_num,review_score
0,0,달바,[물광자차/핑크광채썬] 달바 핑크 톤업 선크림 듀오 기획 (50ml+50ml),"51,000","35,700",선케어,선크림,선크림,"(7,408건)",4.7
1,1,이지듀,[한가인PICK/기미커버개선] 이지듀 기미 앰플 쿠션 15g,"39,800","36,900",선케어,선쿠션,선쿠션,(175건),4.6
2,2,구달,[1+1기획] 구달 맑은 어성초 진정 수분 선크림 50ml 1+1기획,"20,000","17,500",선케어,선크림,선크림,"(28,908건)",4.8
3,3,라운드랩,[1등썬/수분밀착] 라운드랩 자작나무 수분 선크림 50ml 기획 (+클렌저 20ml),"25,000","17,500",선케어,선크림,선크림,"(14,207건)",4.8
4,4,라로슈포제,[2월 올영픽/올영 단독] 라로슈포제 시카플라스트 리페어 선크림 40ml 기획 (+...,"34,000","25,000",선케어,선크림,선크림,(137건),4.7


In [369]:
# 인덱스 순으로 랭킹 컬럼 삽입 - 인덱스 컬럼 변경 및 값 변경
olive_suncare_df.rename(columns={'index':'ranking'}, inplace=True)
olive_suncare_df['ranking'] = olive_suncare_df['ranking'].apply(lambda x : x+1)
olive_suncare_df.head()

,ranking,brand,product,or_price,dis_price,main_class,mid_class,sub_class,review_num,review_score
0,1,달바,[물광자차/핑크광채썬] 달바 핑크 톤업 선크림 듀오 기획 (50ml+50ml),"51,000","35,700",선케어,선크림,선크림,"(7,408건)",4.7
1,2,이지듀,[한가인PICK/기미커버개선] 이지듀 기미 앰플 쿠션 15g,"39,800","36,900",선케어,선쿠션,선쿠션,(175건),4.6
2,3,구달,[1+1기획] 구달 맑은 어성초 진정 수분 선크림 50ml 1+1기획,"20,000","17,500",선케어,선크림,선크림,"(28,908건)",4.8
3,4,라운드랩,[1등썬/수분밀착] 라운드랩 자작나무 수분 선크림 50ml 기획 (+클렌저 20ml),"25,000","17,500",선케어,선크림,선크림,"(14,207건)",4.8
4,5,라로슈포제,[2월 올영픽/올영 단독] 라로슈포제 시카플라스트 리페어 선크림 40ml 기획 (+...,"34,000","25,000",선케어,선크림,선크림,(137건),4.7


In [ ]:
# 원본 데이터 저장
# olive_suncare_df.to_csv("oliveyoung_suncare_ranking.csv", index=False, encoding="utf-8-sig")

## 2. 올리브영 메이크업 크롤링

In [371]:
# 선케어 랭킹 페이지 로드
driver.get("https://www.oliveyoung.co.kr/store/main/getBestList.do?dispCatNo=900000100100001&fltDispCatNo=10000010002&pageIdx=1&rowsPerPage=8&t_page=%EB%9E%AD%ED%82%B9&t_click=%ED%8C%90%EB%A7%A4%EB%9E%AD%ED%82%B9_%EB%A9%94%EC%9D%B4%ED%81%AC%EC%97%85")
time.sleep(3)  # 페이지 로드 대기

In [372]:
# 2. 페이지 스크롤
prev_height = driver.execute_script("return document.body.scrollHeight")

# 1위 - 100위
while True:
    # 스크롤 내리기
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)

    # 새로운 높이 가져오기
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == prev_height:
        break
    prev_height = new_height

In [373]:
# 3-1. 랭킹 페이지 데이터 추출출
titles = driver.find_elements(By.XPATH, '//*[@id="Container"]/div[2]/div[2]/ul/li/div/div/a/span')
products = driver.find_elements(By.XPATH, '//*[@id="Container"]/div[2]/div[2]/ul/li/div/div/a/p')
or_prices = driver.find_elements(By.XPATH, '//*[@id="Container"]/div[2]/div[2]/ul/li/div/p[1]/span[1]/span')
dis_prices = driver.find_elements(By.XPATH, '//*[@id="Container"]/div[2]/div[2]/ul/li/div/p[1]/span[2]/span')

print(len(titles))
print(len(products))
print(len(or_prices))
print(len(dis_prices)) # 세일 안하는 상품들 맞는지 더블 체크 필요

100
100
100
80


In [374]:
# 4-1. 데이터 텍스트로 변환
title_list = [t.text for t in titles]
print(title_list[0:5])

product_list = [p.text for p in products]
print(product_list[0:5])

or_price_list = [op.text for op in or_prices]
print(or_price_list[0:5])

# dis_price_list = [dis.text for dis in dis_prices]
# print(dis_price_list[0:5])

['어뮤즈', '3CE', '롬앤', '에스쁘아', '에스쁘아']
['[신규컬러/단독기획] 어뮤즈 세라믹 스킨 퍼펙터 쿠션 발레리나 에디션 한정 기획 (+리필+퍼프 증정)', '[2월올영픽] 3CE 캐시미어 허그 립스틱 10colors (기획세트 / 단품)', '[2024 어워즈/NEW] 롬앤 더 쥬시 래스팅 틴트 23 Color', '[2월 올영픽/선런칭] 에스쁘아 비벨벳 커버쿠션 SPF42 PA++ (본품+리필) 8 colors + 미니 파우더', '[2월 올영픽/단독기획] 에스쁘아 꾸뛰르 립틴트 글레이즈 20 colors']
['38,000', '22,000', '13,000', '40,000', '22,000']


In [375]:
# 4-1. 할인가 밀림 현상 방지 및 데이터 텍스트 변환
dis_price_list = []

for w in range(1, 26):
  for l in range(1, 5):
    
    try:
      dis = driver.find_element(By.XPATH, f'//*[@id="Container"]/div[2]/div[2]/ul[{w}]/li[{l}]/div/p[1]/span[2]/span')
      if dis.text:
         dis_price_list.append(dis.text)
      else:
        dis_price_list.append('') 
             
    except:
      dis_price_list.append('')
      
print(len(dis_price_list))
print(dis_price_list[:5])

100
['28,500', '16,500', '9,900', '28,900', '16,500']


In [376]:
# 3-2. 상품 상세 페이지 데이터 추출
wait = WebDriverWait(driver, 10)

main_classes = []
mid_classes = []
sub_classes = []
review_nums = []
review_scores = []

for w in range(1, 26):
  for l in range(1, 5):
    detail_page = driver.find_elements(By.XPATH, f'//*[@id="Container"]/div[2]/div[2]/ul[{w}]/li[{l}]/div/a')
    
    if detail_page:
        detail_page[0].click()
    else:
      continue

    main_class = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="midCatNm"]'))).text
    mid_class = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="smlCatNm"]'))).text
    sub_class = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="dtlCatNm"]'))).text 
    num = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="repReview"]/em'))).text
    score = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="repReview"]/b'))).text
  
    main_classes.append(main_class)
    mid_classes.append(mid_class)
    sub_classes.append(sub_class)
    review_nums.append(num)
    review_scores.append(score)

    driver.back()
    wait.until(EC.presence_of_element_located((By.XPATH, f'//*[@id="Container"]/div[2]/div[2]/ul[{w}]/li[{l}]/div/a')))

In [377]:
# 4-2. 데이터 개수 확인
print(len(main_classes))
print(len(mid_classes))
print(len(sub_classes))
print(len(review_nums))
print(len(review_scores))

print(main_classes[0:5])
print(mid_classes[0:5])
print(sub_classes[0:5])
print(review_nums[0:5])
print(review_scores[0:5])

100
100
100
100
100
['메이크업', '메이크업', '메이크업', '메이크업', '메이크업']
['베이스메이크업', '립메이크업', '립메이크업', '베이스메이크업', '립메이크업']
['쿠션', '립스틱', '립틴트', '쿠션', '립틴트']
['(9,122건)', '(1,153건)', '(152,984건)', '(13,489건)', '(8,504건)']
['4.6', '4.7', '4.7', '4.7', '4.8']


In [378]:
# 5. 데이터 저장
olive_makeup_dic = {"brand": title_list, "product": product_list, "or_price": or_price_list, "dis_price": dis_price_list, "main_class": main_classes, "mid_class": mid_classes, "sub_class": sub_classes, "review_num": review_nums, "review_score": review_scores}

In [379]:
# DF 변환
olive_makeup_df = pd.DataFrame.from_dict(olive_makeup_dic, orient='index')
olive_makeup_df = olive_makeup_df.transpose()
olive_makeup_df.head()

,brand,product,or_price,dis_price,main_class,mid_class,sub_class,review_num,review_score
0,어뮤즈,[신규컬러/단독기획] 어뮤즈 세라믹 스킨 퍼펙터 쿠션 발레리나 에디션 한정 기획 (...,"38,000","28,500",메이크업,베이스메이크업,쿠션,"(9,122건)",4.6
1,3CE,[2월올영픽] 3CE 캐시미어 허그 립스틱 10colors (기획세트 / 단품),"22,000","16,500",메이크업,립메이크업,립스틱,"(1,153건)",4.7
2,롬앤,[2024 어워즈/NEW] 롬앤 더 쥬시 래스팅 틴트 23 Color,"13,000","9,900",메이크업,립메이크업,립틴트,"(152,984건)",4.7
3,에스쁘아,[2월 올영픽/선런칭] 에스쁘아 비벨벳 커버쿠션 SPF42 PA++ (본품+리필) ...,"40,000","28,900",메이크업,베이스메이크업,쿠션,"(13,489건)",4.7
4,에스쁘아,[2월 올영픽/단독기획] 에스쁘아 꾸뛰르 립틴트 글레이즈 20 colors,"22,000","16,500",메이크업,립메이크업,립틴트,"(8,504건)",4.8


In [380]:
# 할인가 정상적으로 들어갔는지 확인
print(len(dis_prices))
print(olive_makeup_df.loc[olive_makeup_df['dis_price']=='']['product'].nunique())
olive_makeup_df.loc[olive_makeup_df['dis_price']=='']

80
20


,brand,product,or_price,dis_price,main_class,mid_class,sub_class,review_num,review_score
12,오브제,[2024 어워즈] 오브제 커버로션 50g 어워즈 한정기획 (+미니어처 10g),"29,800",,메이크업,베이스메이크업,BB/CC,"(1,201건)",4.8
14,클리오,[NO.1 아이라이너] 클리오 샤프 쏘 심플 워터프루프 펜슬라이너,"11,000",,메이크업,아이메이크업,아이라이너,"(103,909건)",4.7
16,투크,[2024 어워즈/NEW 컬러 출시] 투크 워터프루프 슬림 아이라이너 16colors,"13,900",,메이크업,아이메이크업,아이라이너,"(30,474건)",4.7
29,하트퍼센트,[NEW/단독기획] 하트퍼센트 도트 온 무드 립펜슬 20colors (기획/단품),"14,000",,메이크업,립메이크업,립스틱,"(27,244건)",4.7
37,정샘물,[NEW] 정샘물 스킨 세팅 베이스 6종,"33,000",,메이크업,베이스메이크업,프라이머/베이스,"(1,890건)",4.6
39,VDL,"VDL 커버스테인 퍼펙팅 파운데이션 30ml (+휴대용파데, 퍼프기획/ 단품)","30,000",,메이크업,베이스메이크업,파운데이션,"(9,338건)",4.7
60,엔트로피 메이크업,[NEW 출시] 엔트로피 메이크업 브로우 컬러 애쉬브라운,"45,000",,메이크업,아이메이크업,아이브로우,"(1,288건)",4.7
63,지베르니,[NEW] 지베르니 플러피 듀얼 블러셔 3 COLORS (LAZY BUNNY) (온),"19,000",,메이크업,베이스메이크업,블러셔,(0건),0
64,페리페라,페리페라 잉크 무드 글로이 틴트,"11,000",,메이크업,립메이크업,립틴트,"(83,546건)",4.7
66,데이지크,[2024어워즈/3년연속수상] 데이지크 섀도우팔레트 24 colors (단품/기획),"34,000",,메이크업,아이메이크업,아이섀도우,"(29,268건)",4.7


In [381]:
# 인덱스 순으로 랭킹 컬럼 삽입 - 인덱스 컬럼 생성
olive_makeup_df.reset_index(inplace=True)
olive_makeup_df.head()

,index,brand,product,or_price,dis_price,main_class,mid_class,sub_class,review_num,review_score
0,0,어뮤즈,[신규컬러/단독기획] 어뮤즈 세라믹 스킨 퍼펙터 쿠션 발레리나 에디션 한정 기획 (...,"38,000","28,500",메이크업,베이스메이크업,쿠션,"(9,122건)",4.6
1,1,3CE,[2월올영픽] 3CE 캐시미어 허그 립스틱 10colors (기획세트 / 단품),"22,000","16,500",메이크업,립메이크업,립스틱,"(1,153건)",4.7
2,2,롬앤,[2024 어워즈/NEW] 롬앤 더 쥬시 래스팅 틴트 23 Color,"13,000","9,900",메이크업,립메이크업,립틴트,"(152,984건)",4.7
3,3,에스쁘아,[2월 올영픽/선런칭] 에스쁘아 비벨벳 커버쿠션 SPF42 PA++ (본품+리필) ...,"40,000","28,900",메이크업,베이스메이크업,쿠션,"(13,489건)",4.7
4,4,에스쁘아,[2월 올영픽/단독기획] 에스쁘아 꾸뛰르 립틴트 글레이즈 20 colors,"22,000","16,500",메이크업,립메이크업,립틴트,"(8,504건)",4.8


In [382]:
# 인덱스 순으로 랭킹 컬럼 삽입 - 인덱스 컬럼 변경 및 값 변경
olive_makeup_df.rename(columns={'index':'ranking'}, inplace=True)
olive_makeup_df['ranking'] = olive_makeup_df['ranking'].apply(lambda x : x+1)
olive_makeup_df.head()

,ranking,brand,product,or_price,dis_price,main_class,mid_class,sub_class,review_num,review_score
0,1,어뮤즈,[신규컬러/단독기획] 어뮤즈 세라믹 스킨 퍼펙터 쿠션 발레리나 에디션 한정 기획 (...,"38,000","28,500",메이크업,베이스메이크업,쿠션,"(9,122건)",4.6
1,2,3CE,[2월올영픽] 3CE 캐시미어 허그 립스틱 10colors (기획세트 / 단품),"22,000","16,500",메이크업,립메이크업,립스틱,"(1,153건)",4.7
2,3,롬앤,[2024 어워즈/NEW] 롬앤 더 쥬시 래스팅 틴트 23 Color,"13,000","9,900",메이크업,립메이크업,립틴트,"(152,984건)",4.7
3,4,에스쁘아,[2월 올영픽/선런칭] 에스쁘아 비벨벳 커버쿠션 SPF42 PA++ (본품+리필) ...,"40,000","28,900",메이크업,베이스메이크업,쿠션,"(13,489건)",4.7
4,5,에스쁘아,[2월 올영픽/단독기획] 에스쁘아 꾸뛰르 립틴트 글레이즈 20 colors,"22,000","16,500",메이크업,립메이크업,립틴트,"(8,504건)",4.8


In [ ]:
# 원본 데이터 저장
# olive_makeup_df.to_csv("oliveyoung_makeup_ranking.csv", index=False, encoding="utf-8-sig")

In [525]:
driver.quit()

# 데이터 전처리

## 데이터셋 공통 전처리_통합 버전

In [3]:
olive_clean_df = pd.read_csv('oliveyoung_cleansing_ranking.csv')
olive_makeup_df = pd.read_csv('oliveyoung_makeup_ranking.csv')
olive_mask_df = pd.read_csv('oliveyoung_mask_ranking.csv')
olive_skincare_df = pd.read_csv('oliveyoung_skincare_ranking.csv')
olive_suncare_df = pd.read_csv('oliveyoung_suncare_ranking.csv')

In [4]:
olive_df = pd.concat([olive_skincare_df, olive_clean_df, olive_mask_df, olive_suncare_df, olive_makeup_df], axis=0, ignore_index=True)
print(olive_df.info())
olive_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   ranking       500 non-null    int64  
 1   brand         500 non-null    object 
 2   product       500 non-null    object 
 3   or_price      500 non-null    object 
 4   dis_price     432 non-null    object 
 5   main_class    500 non-null    object 
 6   mid_class     500 non-null    object 
 7   sub_class     500 non-null    object 
 8   review_num    500 non-null    object 
 9   review_score  500 non-null    float64
dtypes: float64(1), int64(1), object(8)
memory usage: 39.2+ KB
None


,ranking,brand,product,or_price,dis_price,main_class,mid_class,sub_class,review_num,review_score
0,1,에스트라,[2월올영픽/1등크림] 에스트라 아토베리어365 크림 80ML 더블기획 (+세라히알...,"59,400","44,500",스킨케어,크림,크림,"(25,321건)",4.9
1,2,에스트라,[50%용량증정]에스트라 아토베리어365 세라-히알 속수분 앰플 30ML+15ML 기획,"33,000","24,700",스킨케어,에센스/세럼/앰플,에센스/세럼/앰플,"(1,957건)",4.8
2,3,에스트라,[2월올영픽]에스트라 아토베리어365 로션 150ml 리필 기획 (+150ML 리필...,"53,400","39,900",스킨케어,로션,로션,"(2,688건)",4.9
3,4,바이오힐보,[2월 올영픽/PDRN 키트증정] 바이오힐보 프로바이오덤 콜라겐 리모델링 세럼 30...,"49,500","27,300",스킨케어,에센스/세럼/앰플,에센스/세럼/앰플,"(2,226건)",4.7
4,5,구달,[세럼1위/6년연속수상/총 120ml 대용량]구달 청귤 비타C 잡티케어 세럼 70m...,"50,000","32,900",스킨케어,에센스/세럼/앰플,에센스/세럼/앰플,"(28,846건)",4.8


In [251]:
# 1. 상품명 정리 - 구버전버전
'''
for i in range(len(olive_df)):
  name = olive_df['product'][i]

  name_index = name.find(']')
  
  if name_index != -1:
    name_result = name[name_index+1:]
    olive_df.loc[i, 'product'] = name_result
  
  else:
    continue

print(olive_df.loc[:5, 'product'])
'''

"\nfor i in range(len(olive_df)):\n  name = olive_df['product'][i]\n\n  name_index = name.find(']')\n  \n  if name_index != -1:\n    name_result = name[name_index+1:]\n    olive_df.loc[i, 'product'] = name_result\n  \n  else:\n    continue\n\nprint(olive_df.loc[:5, 'product'])\n"

In [252]:
a = '테스트용'

a_253 = a.split(']')
max_a253 = len(a_253)-1

print(a_253[0], len(a_253))


b = olive_df['product'][0]

b_253 = b.split(']')
max_b253 = len(b_253)-1

print(b_253[0][0], '/', b_253, b_253[max_b253], '/', len(b_253))


c = ' 테스트용[ dddd]'

c_253 = c.split(']')
max_c253 = len(c_253)-1
c_result = c_253[0]
c_second = c_result.split('[')
c_result2 = c_second[0]

print(f"'{c_253[0].strip()}'", '/', c_253, c_253[max_c253], '/', len(c_253), c_result2)


d = olive_df['product'][60]

d_253 = d.split(']')
max_d253 = len(d_253)-1

print(d_253, d_253[max_d253], '/', len(d_253))

테스트용 1
[ / ['[2월올영픽/1등크림', ' 에스트라 아토베리어365 크림 80ML 더블기획 (+세라히알 앰플 7ML+에이시카 세럼 3ML)']  에스트라 아토베리어365 크림 80ML 더블기획 (+세라히알 앰플 7ML+에이시카 세럼 3ML) / 2
'테스트용[ dddd' / [' 테스트용[ dddd', '']  / 2  테스트용
['[아이소이x망그러진곰', '[대용량', ' 아이소이 모이스춰 닥터 장수진 수분크림100ml 한정기획(+키링&장바구니증정)']  아이소이 모이스춰 닥터 장수진 수분크림100ml 한정기획(+키링&장바구니증정) / 3


- 경우의 수
1) [] 가 맨 앞에 있을 경우('[', '상품명') -> 리스트의 마지막 글자 사용 name_result = name_list[i][max_i] -> name_result.strip() => 상품명
3) [] 안이 상품명일 경우('[상품명','') -> name_result = name_list[i][0] -> name_result.strip('[') & strip() => 상품명

2) [] 가 맨 뒤에 있을 경우('상품명[~', '') -> 리스트의 첫 글자 사용 name_result = name_list[i][0] => 상품명[~

4) [] 가 앞뒤로 있을 경우('[~','상품명[~','') -> name_result = name_list[i][max_1-1] => 상품명[~

5) [] 가 아예 없는 경우 

6) []가 두번 연속 있을 경우 

In [5]:
# 1. 상품명 정리 - 신버전 
for i in range(len(olive_df)):
  
  name_list = olive_df['product'][i].split(']')
  max_i = len(name_list)-1
  
  if (len(name_list) == 2):
    if (name_list[0][0] == '['):
      if name_list[1] == '': # 3)의 경우
        name_result = name_list[0]
        name_result = name_result.strip('[')
        name_result = name_result.strip()
        olive_df['product'][i] = name_result
      else: # 1)의 경우 
        name_result = name_list[1]
        name_result = name_result.strip()
        olive_df['product'][i] = name_result
    
    else: # 2)의 경우
      name_result = name_list[0]
      second_list = name_result.split('[')
      name_result2 = second_list[0]
      olive_df['product'][i] = name_result2
  
  elif (len(name_list)==1): # 5)의 경우 
    name_result = name_list[0]
    olive_df['product'][i] = name_result
  
  else: # 길이가 3일 경우
    if name_list[2] == '': # 4)의 경우 
      name_result = name_list[1]
      second_list = name_result.split('[')
      name_result2 = second_list[0]
      olive_df['product'][i] = name_result2
    
    else: # 6의 경우 
      name_result = name_list[2]
      name_result = name_result.strip()
      olive_df['product'][i] = name_result

print(olive_df['product'].unique())     

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_9232\2916843692.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  olive_df['product'][i] = name_result
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_9232\2916843692.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  olive_df['product'][i] = name_result
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_9232\2916843692.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus

['에스트라 아토베리어365 크림 80ML 더블기획 (+세라히알 앰플 7ML+에이시카 세럼 3ML)'
 '에스트라 아토베리어365 세라-히알 속수분 앰플 30ML+15ML 기획'
 '에스트라 아토베리어365 로션 150ml 리필 기획 (+150ML 리필 파우치 + 에이시카 세럼 3ML)'
 '바이오힐보 프로바이오덤 콜라겐 리모델링 세럼 30ml 리필 기획 (+프로그램14ml)'
 '구달 청귤 비타C 잡티케어 세럼 70ml 대용량 어워즈 한정기획'
 '바이오더마 시카비오 크림+ 100ml 기획 (+40ml 증정)'
 '토리든 다이브인 저분자 히알루론산 세럼 50ml 리필기획(+리필팩 50ml)' '에스트라 아토베리어365 크림미스트 120ml'
 '닥터자르트 시카페어 인텐시브 수딩 리페어 세럼 30ml 1+1 기획'
 '라로슈포제 시카플라스트 밤 B5+ 200ml 기획 (+시카PRO마스크 1매)'
 '차앤박 더마앤서 액티브 부스트 PDRN앰플 30ml 기획 (+15ml 증정)'
 '웰라쥬 리얼 히알루로닉 블루 100 앰플 75ml 1+1 기획' '바이오더마 시카비오 포마드 100ml(리페어 리치 밤)'
 '바이오힐보 프로바이오덤 콜라겐 리모델링 세럼 30ml+30ml 더블기획' '셀라딕스 세범 리밸런싱 RX 131 앰플'
 '차앤박 프로폴리스 에너지 액티브 앰플 30ml 트리플 기획'
 '에스네이처 아쿠아 스쿠알란 수분크림 60ml 더블 기획(60ml+60ml)'
 '닥터지 레드 블레미쉬 클리어 수딩 크림 기획세트 (100ml/150ml구성)'
 '바이오더마 하이드라비오 에센스 로션 200ml(화장솜 증정)'
 '라네즈 크림스킨 170ml 리필기획 (+170ml 리필+미스트펌프 증정)'
 '바이오힐보 프로바이오덤 3D 리프팅 크림 50ml 리필기획 (+윤곽마스크 2매)(올리지오 콜라보)'
 '메디큐브 연어 PDRN 핑크 앰플 더블기획 (30ml+30ml)'
 '라로슈포제 시카플라스트 크림 B5 100ml 기획 (+시카토너 50ml+시카크림 3ml 증정)'
 '파

In [6]:
# 결측치 더블체크
print(olive_df[olive_df['product'] == ''])
print(olive_df[olive_df['product'] == ' '])

Empty DataFrame
Columns: [ranking, brand, product, or_price, dis_price, main_class, mid_class, sub_class, review_num, review_score]
Index: []
Empty DataFrame
Columns: [ranking, brand, product, or_price, dis_price, main_class, mid_class, sub_class, review_num, review_score]
Index: []


In [7]:
# 2. 리뷰 수 서식 지우기

for i in range(len(olive_df)):
  name = olive_df['review_num'][i]

  name_result = name.strip('(''건'')')
  name_result = name_result.replace(',', '')
  olive_df.loc[i, 'review_num'] = name_result

print(olive_df.loc[:5, 'review_num'])

0    25321
1     1957
2     2688
3     2226
4    28846
5     3561
Name: review_num, dtype: object


In [8]:
# 결측값이 float 형태로 읽히는 것을 방지하기 위해 결측값 공백으로 대체
olive_df['dis_price'].fillna('', inplace=True)

olive_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   ranking       500 non-null    int64  
 1   brand         500 non-null    object 
 2   product       500 non-null    object 
 3   or_price      500 non-null    object 
 4   dis_price     500 non-null    object 
 5   main_class    500 non-null    object 
 6   mid_class     500 non-null    object 
 7   sub_class     500 non-null    object 
 8   review_num    500 non-null    object 
 9   review_score  500 non-null    float64
dtypes: float64(1), int64(1), object(8)
memory usage: 39.2+ KB


In [9]:
# 3. 컬럼 타입 변경 : score(float), review_num(int), price(num)
# 1) price 컬럼의 쉼표 없애기

for i in range(len(olive_df)):
  or_p = olive_df['or_price'][i]
  dis_p = olive_df['dis_price'][i]
  
  or_result = or_p.replace(',', '')
  dis_result = dis_p.replace(',', '')
  
  olive_df.loc[i, 'or_price'] = or_result
  olive_df.loc[i, 'dis_price'] = dis_result

print(olive_df.loc[:5, ['or_price', 'dis_price', 'review_num']])

  or_price dis_price review_num
0    59400     44500      25321
1    33000     24700       1957
2    53400     39900       2688
3    49500     27300       2226
4    50000     32900      28846
5    43000     33900       3561


In [10]:
# 3. 컬럼 타입 변경 : score(float), review_num(int), price(num)
# 2) 컬럼 타입 변경

olive_df[['or_price', 'dis_price', 'review_num', 'review_score']] = olive_df[['or_price', 'dis_price', 'review_num', 'review_score']].apply(pd.to_numeric)
print(olive_df[['or_price', 'dis_price', 'review_num', 'review_score']].dtypes)
print(olive_df.loc[:5, ['or_price', 'dis_price', 'review_num', 'review_score']])

or_price          int64
dis_price       float64
review_num        int64
review_score    float64
dtype: object
   or_price  dis_price  review_num  review_score
0     59400    44500.0       25321           4.9
1     33000    24700.0        1957           4.8
2     53400    39900.0        2688           4.9
3     49500    27300.0        2226           4.7
4     50000    32900.0       28846           4.8
5     43000    33900.0        3561           4.8


In [11]:
# 4. 플랫폼 컬럼 삽입 : df['플랫폼'] = '올리브영'
olive_df['플랫폼'] = '올리브영'
print(olive_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   ranking       500 non-null    int64  
 1   brand         500 non-null    object 
 2   product       500 non-null    object 
 3   or_price      500 non-null    int64  
 4   dis_price     432 non-null    float64
 5   main_class    500 non-null    object 
 6   mid_class     500 non-null    object 
 7   sub_class     500 non-null    object 
 8   review_num    500 non-null    int64  
 9   review_score  500 non-null    float64
 10  플랫폼           500 non-null    object 
dtypes: float64(2), int64(3), object(6)
memory usage: 43.1+ KB
None


In [12]:
# 5. 컬럼명 및 순서 변경 : 플랫폼-브랜드-제품명-대분류_순위-대분류-중분류-소분류-평점-후기 수-가격
# 1) 컬럼명 변경
olive_df.columns = ['중분류_순위', '브랜드', '제품명', '정가', '할인가', '대분류', '중분류', '소분류', '후기 수', '평점', '플랫폼']
print(olive_df.columns)

Index(['중분류_순위', '브랜드', '제품명', '정가', '할인가', '대분류', '중분류', '소분류', '후기 수', '평점',
       '플랫폼'],
      dtype='object')


In [13]:
olive_df = olive_df.reindex(['플랫폼', '브랜드', '제품명', '중분류_순위', '대분류', '중분류', '소분류', '평점', '후기 수', '정가', '할인가'], axis=1)

print(olive_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 11 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   플랫폼     500 non-null    object 
 1   브랜드     500 non-null    object 
 2   제품명     500 non-null    object 
 3   중분류_순위  500 non-null    int64  
 4   대분류     500 non-null    object 
 5   중분류     500 non-null    object 
 6   소분류     500 non-null    object 
 7   평점      500 non-null    float64
 8   후기 수    500 non-null    int64  
 9   정가      500 non-null    int64  
 10  할인가     432 non-null    float64
dtypes: float64(2), int64(3), object(6)
memory usage: 43.1+ KB
None


In [262]:
# 데이터셋 저장
olive_df.to_csv('통합_올리브영_데이터.csv', index=False)

In [263]:
check_df = pd.read_csv('통합_올리브영_데이터.csv')
check_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 11 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   플랫폼     500 non-null    object 
 1   브랜드     500 non-null    object 
 2   제품명     500 non-null    object 
 3   중분류_순위  500 non-null    int64  
 4   대분류     500 non-null    object 
 5   중분류     500 non-null    object 
 6   소분류     500 non-null    object 
 7   평점      500 non-null    float64
 8   후기 수    500 non-null    int64  
 9   정가      500 non-null    int64  
 10  할인가     432 non-null    float64
dtypes: float64(2), int64(3), object(6)
memory usage: 43.1+ KB


## 데이터셋 공통 전처리_개별 버전

- 상품명 정리 : 맨 앞 대괄호 수식어 삭제
- 리뷰 수 서식 지우기
- 숫자 형식 컬럼 타입 문자에서 숫자로 변경

In [14]:
# 데이터셋 정보 확인
oliveyoung = [olive_skincare_df, olive_mask_df, olive_clean_df, olive_suncare_df, olive_makeup_df]

for o in oliveyoung:
  print(o.info()) # 숫자 형식 컬럼 모두 문자열
  print(o.isnull().sum()) # null 값 없음

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   ranking       100 non-null    int64  
 1   brand         100 non-null    object 
 2   product       100 non-null    object 
 3   or_price      100 non-null    object 
 4   dis_price     97 non-null     object 
 5   main_class    100 non-null    object 
 6   mid_class     100 non-null    object 
 7   sub_class     100 non-null    object 
 8   review_num    100 non-null    object 
 9   review_score  100 non-null    float64
dtypes: float64(1), int64(1), object(8)
memory usage: 7.9+ KB
None
ranking         0
brand           0
product         0
or_price        0
dis_price       3
main_class      0
mid_class       0
sub_class       0
review_num      0
review_score    0
dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 10 columns):
 #   Column     

In [15]:
# 데이터셋 복제
olive_skincare_df2 = olive_skincare_df.copy(deep=True)
olive_mask_df2 = olive_mask_df.copy(deep=True)
olive_clean_df2 = olive_clean_df.copy(deep=True)
olive_suncare_df2 = olive_suncare_df.copy(deep=True)
olive_makeup_df2 = olive_makeup_df.copy(deep=True)

# 복제 데이터셋 리스트 생성
oliveyoung_copy = [olive_skincare_df2, olive_mask_df2, olive_clean_df2, olive_suncare_df2, olive_makeup_df2]

In [16]:
# 1. 상품명 정리
for o in oliveyoung_copy:
  for i in range(0, 100):
    name = o['product'][i]
  
    name_index = name.find(']')
    
    if name_index != -1:
      name_result = name[name_index+1:]
      o.loc[i, 'product'] = name_result
    
    else:
      continue
  
  print(o.loc[:2, 'product'])

0     에스트라 아토베리어365 크림 80ML 더블기획 (+세라히알 앰플 7ML+에이시카...
1              에스트라 아토베리어365 세라-히알 속수분 앰플 30ML+15ML 기획
2    에스트라 아토베리어365 로션 150ml 리필 기획 (+150ML 리필 파우치 + ...
Name: product, dtype: object
0           더마토리 히알루론 수분샘 엠보 패드 60매+10매 기획
1       더마토리 시카 엑소좀 흔적 진정 거즈 패드 70매+10매 기획
2     더마토리 살리시닉 포어 클리어 블랙 패드 70매+10매 기획 세트
Name: product, dtype: object
0       휩드 비건 팩클렌저 130g 기획/단품 (무화버터/머그트리/유자몽)
1                 마녀공장 퓨어 클렌징 오일 200mlX2 더블기획
2    바이오더마 센시비오 H2O 850ml 기획 (+K2 콜라보 폴딩 에코백)
Name: product, dtype: object
0     달바 핑크 톤업 선크림 듀오 기획 (50ml+50ml)
1                   이지듀 기미 앰플 쿠션 15g
2     구달 맑은 어성초 진정 수분 선크림 50ml 1+1기획
Name: product, dtype: object
0     어뮤즈 세라믹 스킨 퍼펙터 쿠션 발레리나 에디션 한정 기획 (+리필+퍼프 증정)
1             3CE 캐시미어 허그 립스틱 10colors (기획세트 / 단품)
2                          롬앤 더 쥬시 래스팅 틴트 23 Color
Name: product, dtype: object


In [17]:
# 2. 리뷰 수 서식 지우기
for o in oliveyoung_copy:
  for i in range(0, 100):
    name = o['review_num'][i]
  
    name_result = name.strip('(''건'')')
    name_result = name_result.replace(',', '')
    o.loc[i, 'review_num'] = name_result

  print(o.loc[:2, 'review_num'])

0    25321
1     1957
2     2688
Name: review_num, dtype: object
0      8
1     14
2    143
Name: review_num, dtype: object
0     2162
1    13929
2    82075
Name: review_num, dtype: object
0     7408
1      175
2    28908
Name: review_num, dtype: object
0      9122
1      1153
2    152984
Name: review_num, dtype: object


In [18]:
# 3. 컬럼 타입 변경 : score(float), review_num(int), price(num)
# 1) price 컬럼의 쉼표 없애기

for o in oliveyoung_copy:
  for i in range(0, 100):
    or_p = o['or_price'][i]
    dis_p = o['dis_price'][i]
    
    or_result = or_p.replace(',', '')
    dis_result = dis_p.replace(',', '')
    
    o.loc[i, 'or_price'] = or_result
    o.loc[i, 'dis_price'] = dis_result

  print(o.loc[:2, ['or_price', 'dis_price', 'review_num']])

AttributeError: 'float' object has no attribute 'replace'

In [ ]:
# 3. 컬럼 타입 변경 : score(float), review_num(int), price(num)
# 2) 컬럼 타입 변경

for o in oliveyoung_copy:
  o[['or_price', 'dis_price', 'review_num', 'review_score']] = o[['or_price', 'dis_price', 'review_num', 'review_score']].apply(pd.to_numeric)
  print(o[['or_price', 'dis_price', 'review_num', 'review_score']].dtypes)
  print(o.loc[:5, ['or_price', 'dis_price', 'review_num', 'review_score']])

or_price          int64
dis_price       float64
review_num        int64
review_score    float64
dtype: object
   or_price  dis_price  review_num  review_score
0     59400    44500.0       25321           4.9
1     33000    24700.0        1957           4.8
2     53400    39900.0        2688           4.9
3     49500    27300.0        2226           4.7
4     50000    32900.0       28846           4.8
5     43000    33900.0        3561           4.8
or_price          int64
dis_price       float64
review_num        int64
review_score    float64
dtype: object
   or_price  dis_price  review_num  review_score
0     26000    17600.0           8           4.6
1     26000    17600.0          14           4.9
2     26000    17600.0         143           4.9
3     39900    28900.0       26581           4.7
4      2000     1000.0      229962           4.9
5     20000    10000.0       29652           4.9
or_price          int64
dis_price       float64
review_num        int64
review_score    float6

In [ ]:
# 4. 플랫폼 컬럼 삽입 : df['플랫폼'] = '올리브영'
for o in oliveyoung_copy:
  o['플랫폼'] = '올리브영'
  print(o.iloc[:1, -3:])

   review_num  review_score   플랫폼
0       25321           4.9  올리브영
   review_num  review_score   플랫폼
0           8           4.6  올리브영
   review_num  review_score   플랫폼
0        2162           4.8  올리브영
   review_num  review_score   플랫폼
0        7408           4.7  올리브영
   review_num  review_score   플랫폼
0        9122           4.6  올리브영


In [ ]:
# 5. 컬럼명 및 순서 변경 : 플랫폼-브랜드-제품명-대분류_순위-대분류-중분류-소분류-평점-후기 수-가격
# 1) 컬럼명 변경
for o in oliveyoung_copy:
  o.columns = ['중분류_순위', '브랜드', '제품명', '정가', '할인가', '대분류', '중분류', '소분류', '후기 수', '평점', '플랫폼']
  print(o.columns)

Index(['대분류_순위', '브랜드', '제품명', '정가', '할인가', '대분류', '중분류', '소분류', '후기 수', '평점',
       '플랫폼'],
      dtype='object')
Index(['대분류_순위', '브랜드', '제품명', '정가', '할인가', '대분류', '중분류', '소분류', '후기 수', '평점',
       '플랫폼'],
      dtype='object')
Index(['대분류_순위', '브랜드', '제품명', '정가', '할인가', '대분류', '중분류', '소분류', '후기 수', '평점',
       '플랫폼'],
      dtype='object')
Index(['대분류_순위', '브랜드', '제품명', '정가', '할인가', '대분류', '중분류', '소분류', '후기 수', '평점',
       '플랫폼'],
      dtype='object')
Index(['대분류_순위', '브랜드', '제품명', '정가', '할인가', '대분류', '중분류', '소분류', '후기 수', '평점',
       '플랫폼'],
      dtype='object')


In [ ]:
# 5. 컬럼명 및 순서 변경 : 플랫폼-브랜드-제품명-중분류_순위-대분류-중분류-소분류-평점-후기 수-가격
# 2) 컬럼 순서 변경

for i in range(len(oliveyoung_copy)):
  oliveyoung_copy[i] = oliveyoung_copy[i].reindex(['플랫폼', '브랜드', '제품명', '중분류_순위', '대분류', '중분류', '소분류', '평점', '후기 수', '정가', '할인가'], axis=1)

  print(oliveyoung_copy[i].head())

    플랫폼    브랜드                                                제품명  대분류_순위  \
0  올리브영   에스트라   에스트라 아토베리어365 크림 80ML 더블기획 (+세라히알 앰플 7ML+에이시카...       1   
1  올리브영   에스트라            에스트라 아토베리어365 세라-히알 속수분 앰플 30ML+15ML 기획       2   
2  올리브영   에스트라  에스트라 아토베리어365 로션 150ml 리필 기획 (+150ML 리필 파우치 + ...       3   
3  올리브영  바이오힐보    바이오힐보 프로바이오덤 콜라겐 리모델링 세럼 30ml 리필 기획 (+프로그램14ml)       4   
4  올리브영     구달                구달 청귤 비타C 잡티케어 세럼 70ml 대용량 어워즈 한정기획       5   

    대분류        중분류        소분류   평점   후기 수     정가      할인가  
0  스킨케어         크림         크림  4.9  25321  59400  44500.0  
1  스킨케어  에센스/세럼/앰플  에센스/세럼/앰플  4.8   1957  33000  24700.0  
2  스킨케어         로션         로션  4.9   2688  53400  39900.0  
3  스킨케어  에센스/세럼/앰플  에센스/세럼/앰플  4.7   2226  49500  27300.0  
4  스킨케어  에센스/세럼/앰플  에센스/세럼/앰플  4.8  28846  50000  32900.0  
    플랫폼   브랜드                                    제품명  대분류_순위   대분류  중분류  소분류  \
0  올리브영  더마토리         더마토리 히알루론 수분샘 엠보 패드 60매+10매 기획       1  마스크팩   패드   패드   
1  올리브영  더마토리    

In [ ]:
olive_skincare_df2 = oliveyoung_copy[0]
olive_skincare_df2.head()

,플랫폼,브랜드,제품명,대분류_순위,대분류,중분류,소분류,평점,후기 수,정가,할인가
0,올리브영,에스트라,에스트라 아토베리어365 크림 80ML 더블기획 (+세라히알 앰플 7ML+에이시카...,1,스킨케어,크림,크림,4.9,25321,59400,44500.0
1,올리브영,에스트라,에스트라 아토베리어365 세라-히알 속수분 앰플 30ML+15ML 기획,2,스킨케어,에센스/세럼/앰플,에센스/세럼/앰플,4.8,1957,33000,24700.0
2,올리브영,에스트라,에스트라 아토베리어365 로션 150ml 리필 기획 (+150ML 리필 파우치 + ...,3,스킨케어,로션,로션,4.9,2688,53400,39900.0
3,올리브영,바이오힐보,바이오힐보 프로바이오덤 콜라겐 리모델링 세럼 30ml 리필 기획 (+프로그램14ml),4,스킨케어,에센스/세럼/앰플,에센스/세럼/앰플,4.7,2226,49500,27300.0
4,올리브영,구달,구달 청귤 비타C 잡티케어 세럼 70ml 대용량 어워즈 한정기획,5,스킨케어,에센스/세럼/앰플,에센스/세럼/앰플,4.8,28846,50000,32900.0


In [ ]:
olive_mask_df2 = oliveyoung_copy[1]
olive_mask_df2.head()

,플랫폼,브랜드,제품명,대분류_순위,대분류,중분류,소분류,평점,후기 수,정가,할인가
0,올리브영,더마토리,더마토리 히알루론 수분샘 엠보 패드 60매+10매 기획,1,마스크팩,패드,패드,4.6,8,26000,17600.0
1,올리브영,더마토리,더마토리 시카 엑소좀 흔적 진정 거즈 패드 70매+10매 기획,2,마스크팩,패드,패드,4.9,14,26000,17600.0
2,올리브영,더마토리,더마토리 살리시닉 포어 클리어 블랙 패드 70매+10매 기획 세트,3,마스크팩,패드,패드,4.9,143,26000,17600.0
3,올리브영,메디힐,메디힐 네모패드 100+100매 더블기획 7종 골라담기,4,마스크팩,패드,패드,4.7,26581,39900,28900.0
4,올리브영,메디힐,메디힐 에센셜 마스크팩 1매 고기능 7종 택1,5,마스크팩,시트팩,시트팩,4.9,229962,2000,1000.0


In [ ]:
olive_clean_df2 = oliveyoung_copy[2]
olive_clean_df2.head()

,플랫폼,브랜드,제품명,대분류_순위,대분류,중분류,소분류,평점,후기 수,정가,할인가
0,올리브영,휩드,휩드 비건 팩클렌저 130g 기획/단품 (무화버터/머그트리/유자몽),1,클렌징,클렌징폼/젤,클렌징폼/젤,4.8,2162,29000,22900.0
1,올리브영,마녀공장,마녀공장 퓨어 클렌징 오일 200mlX2 더블기획,2,클렌징,오일/밤,클렌징오일,4.9,13929,37000,25900.0
2,올리브영,바이오더마,바이오더마 센시비오 H2O 850ml 기획 (+K2 콜라보 폴딩 에코백),3,클렌징,워터/밀크,클렌징워터,4.9,82075,37500,22900.0
3,올리브영,비플레인,비플레인 녹두 약산성 클렌징폼 160ml + 40ml 기획,4,클렌징,클렌징폼/젤,클렌징폼/젤,4.8,10743,21900,18600.0
4,올리브영,바닐라코,바닐라코 클린잇제로 오리지널 클렌징밤 100ml 기획 (+미니어처키트 7ml*4개입),5,클렌징,오일/밤,클렌징밤,4.9,4121,20000,14800.0


In [ ]:
olive_suncare_df2 = oliveyoung_copy[3]
olive_suncare_df2.head()

,플랫폼,브랜드,제품명,대분류_순위,대분류,중분류,소분류,평점,후기 수,정가,할인가
0,올리브영,달바,달바 핑크 톤업 선크림 듀오 기획 (50ml+50ml),1,선케어,선크림,선크림,4.7,7408,51000,35700.0
1,올리브영,이지듀,이지듀 기미 앰플 쿠션 15g,2,선케어,선쿠션,선쿠션,4.6,175,39800,36900.0
2,올리브영,구달,구달 맑은 어성초 진정 수분 선크림 50ml 1+1기획,3,선케어,선크림,선크림,4.8,28908,20000,17500.0
3,올리브영,라운드랩,라운드랩 자작나무 수분 선크림 50ml 기획 (+클렌저 20ml),4,선케어,선크림,선크림,4.8,14207,25000,17500.0
4,올리브영,라로슈포제,라로슈포제 시카플라스트 리페어 선크림 40ml 기획 (+시카토너50ml+시카밤3m...,5,선케어,선크림,선크림,4.7,137,34000,25000.0


In [ ]:
olive_makeup_df2 = oliveyoung_copy[4]
olive_makeup_df2.head()

,플랫폼,브랜드,제품명,대분류_순위,대분류,중분류,소분류,평점,후기 수,정가,할인가
0,올리브영,어뮤즈,어뮤즈 세라믹 스킨 퍼펙터 쿠션 발레리나 에디션 한정 기획 (+리필+퍼프 증정),1,메이크업,베이스메이크업,쿠션,4.6,9122,38000,28500.0
1,올리브영,3CE,3CE 캐시미어 허그 립스틱 10colors (기획세트 / 단품),2,메이크업,립메이크업,립스틱,4.7,1153,22000,16500.0
2,올리브영,롬앤,롬앤 더 쥬시 래스팅 틴트 23 Color,3,메이크업,립메이크업,립틴트,4.7,152984,13000,9900.0
3,올리브영,에스쁘아,에스쁘아 비벨벳 커버쿠션 SPF42 PA++ (본품+리필) 8 colors + 미...,4,메이크업,베이스메이크업,쿠션,4.7,13489,40000,28900.0
4,올리브영,에스쁘아,에스쁘아 꾸뛰르 립틴트 글레이즈 20 colors,5,메이크업,립메이크업,립틴트,4.8,8504,22000,16500.0


In [ ]:
# 6. 전처리 결과 파일 저장
'''
olive_skincare_df2.to_csv('olive_skincare_df2.csv', index=False, encoding="utf-8-sig")
olive_clean_df2.to_csv('olive_clean_df2.csv', index=False, encoding="utf-8-sig")
olive_mask_df2.to_csv('olive_mask_df2.csv', index=False, encoding="utf-8-sig")
olive_suncare_df2.to_csv('olive_suncare_df2.csv', index=False, encoding="utf-8-sig")
olive_makeup_df2.to_csv('olive_makeup_df2.csv', index=False, encoding="utf-8-sig")
'''

# 올리브영 데이터셋 병합 및 저장

In [ ]:
import datetime

now = datetime.datetime.now()
print(now)

In [ ]:
# 데이터 병합 
oliveyoung_ranking = pd.concat([olive_skincare_df2, olive_clean_df2, olive_mask_df2, olive_suncare_df2, olive_makeup_df2], axis=0, ignore_index=True)
oliveyoung_ranking.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 11 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   플랫폼     500 non-null    object 
 1   브랜드     500 non-null    object 
 2   제품명     500 non-null    object 
 3   대분류_순위  500 non-null    int64  
 4   대분류     500 non-null    object 
 5   중분류     500 non-null    object 
 6   소분류     500 non-null    object 
 7   평점      500 non-null    float64
 8   후기 수    500 non-null    int64  
 9   정가      500 non-null    int64  
 10  할인가     432 non-null    float64
dtypes: float64(2), int64(3), object(6)
memory usage: 43.1+ KB


In [ ]:
# 데이터 저장 
# file_name = f"{now.strftime('%Y-%m-%d')}_oliveyoung_skincare_ranking.csv"
# oliveyoung_ranking.to_csv(file_name, index=False, encoding="utf-8-sig")

In [ ]:
'''
# 스킨케어
file_name = f"{now.strftime('%Y-%m-%d')}_oliveyoung_skincare_ranking.csv"
olive_skincare_df.to_csv(file_name, index=False, encoding="utf-8-sig")

# 마스크팩
file_name = f"{now.strftime('%Y-%m-%d')}_oliveyoung_mask_ranking.csv"
olive_mask_df.to_csv(file_name, index=False, encoding="utf-8-sig")

# 클렌징 
file_name = f"{now.strftime('%Y-%m-%d')}_oliveyoung_cleansing_ranking.csv"
olive_clean_df.to_csv(file_name, index=False, encoding="utf-8-sig")

# 선케어 
file_name = f"{now.strftime('%Y-%m-%d')}_oliveyoung_suncare_ranking.csv"
olive_suncare_df.to_csv(file_name, index=False, encoding="utf-8-sig")

# 메이크업 
file_name = f"{now.strftime('%Y-%m-%d')}_oliveyoung_makeup_ranking.csv"
olive_makeup_df.to_csv(file_name, index=False, encoding="utf-8-sig")
'''

# 올리브영 데이터셋 병합 재확인

In [14]:
olive_clean_df2 = pd.read_csv('olive_clean_df2.csv')
olive_makeup_df2 = pd.read_csv('olive_makeup_df2.csv')
olive_mask_df2 = pd.read_csv('olive_mask_df2.csv')
olive_skincare_df2 = pd.read_csv('olive_skincare_df2.csv')
olive_suncare_df2 = pd.read_csv('olive_suncare_df2.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'olive_clean_df2.csv'

In [6]:
oliveyoung_ranking = pd.concat([olive_skincare_df2, olive_clean_df2, olive_mask_df2, olive_suncare_df2, olive_makeup_df2], axis=0, ignore_index=True)
oliveyoung_ranking.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 11 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   플랫폼     500 non-null    object 
 1   브랜드     500 non-null    object 
 2   제품명     498 non-null    object 
 3   대분류_순위  500 non-null    int64  
 4   대분류     500 non-null    object 
 5   중분류     500 non-null    object 
 6   소분류     500 non-null    object 
 7   평점      500 non-null    float64
 8   후기 수    500 non-null    int64  
 9   정가      500 non-null    int64  
 10  할인가     432 non-null    float64
dtypes: float64(2), int64(3), object(6)
memory usage: 43.1+ KB


In [ ]:
oliveyoung_ranking[oliveyoung_ranking['제품명'].isnull()]

# 후기수=1571 -> 라로슈포제 유비데아 안뗄리오스 틴티드 크림 [톤보정]
# 후기수=1534 -> 라로슈포제 유비데아 안뗄리오스 멜트인 크림 [수분]

,플랫폼,브랜드,제품명,대분류_순위,대분류,중분류,소분류,평점,후기 수,정가,할인가
360,올리브영,라로슈포제,NaN,61,선케어,선크림,선크림,4.7,1571,37000,NaN
399,올리브영,라로슈포제,NaN,100,선케어,선크림,선크림,4.8,1534,37000,NaN


In [11]:
olive_clean_ranking = pd.read_csv('oliveyoung_cleansing_ranking.csv')
olive_makeup_ranking = pd.read_csv('oliveyoung_makeup_ranking.csv')
olive_mask_ranking = pd.read_csv('oliveyoung_mask_ranking.csv')
olive_skincare_ranking = pd.read_csv('oliveyoung_skincare_ranking.csv')
olive_suncare_ranking = pd.read_csv('oliveyoung_suncare_ranking.csv')

In [12]:
olive_ranking = pd.concat([olive_skincare_ranking, olive_clean_ranking, olive_mask_ranking, olive_suncare_ranking, olive_makeup_ranking], axis=0, ignore_index=True)
olive_ranking.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   ranking       500 non-null    int64  
 1   brand         500 non-null    object 
 2   product       500 non-null    object 
 3   or_price      500 non-null    object 
 4   dis_price     432 non-null    object 
 5   main_class    500 non-null    object 
 6   mid_class     500 non-null    object 
 7   sub_class     500 non-null    object 
 8   review_num    500 non-null    object 
 9   review_score  500 non-null    float64
dtypes: float64(1), int64(1), object(8)
memory usage: 39.2+ KB
